In [4]:
from google.colab import files

# Execute e selecione o arquivo .json correto do seu computador
uploaded = files.upload()


Saving applicants.json to applicants.json
Saving vagas.json to vagas.json
Saving prospects.json to prospects.json


In [5]:
pip install scikit-learn==1.6.1

In [6]:
import pandas as pd
import json

# --- ETAPA 1: Transformar 'aplicacoes.json' ---
with open('applicants.json', 'r', encoding='utf-8') as f:
    aplicacoes_raw = json.load(f)

aplicacoes_df = pd.json_normalize(aplicacoes_raw).T.reset_index()
aplicacoes_df[['id_aplicacao', 'campo']] = aplicacoes_df['index'].str.extract(r"^(\d+)\.(.*)")
aplicacoes_df = aplicacoes_df.pivot(index='id_aplicacao', columns='campo', values=0).reset_index()

# --- ETAPA 2: Transformar 'vagas.json' ---
with open('vagas.json', 'r', encoding='utf-8') as f:
    vagas_raw = json.load(f)

vagas_df = pd.json_normalize(vagas_raw).T.reset_index()
vagas_df[['id_vaga', 'campo']] = vagas_df['index'].str.extract(r"^(\d+)\.(.*)")
vagas_df = vagas_df.pivot(index='id_vaga', columns='campo', values=0).reset_index()

# --- ETAPA 3: Tentar detectar automaticamente a coluna de vínculo ---
possiveis_chaves_aplicacao = [col for col in aplicacoes_df.columns if 'vaga' in col.lower() or 'codigo' in col.lower()]
possiveis_chaves_vaga = [col for col in vagas_df.columns if 'id' in col.lower() or 'codigo' in col.lower() or 'vaga' in col.lower()]

print("🔍 Possíveis chaves em 'aplicacoes':", possiveis_chaves_aplicacao)
print("🔍 Possíveis chaves em 'vagas':", possiveis_chaves_vaga)

# Tentar encontrar uma chave em comum (valores coincidentes)
chave_encontrada = None
for col_ap in possiveis_chaves_aplicacao:
    for col_vg in possiveis_chaves_vaga:
        if aplicacoes_df[col_ap].isin(vagas_df[col_vg]).any():
            chave_encontrada = (col_ap, col_vg)
            break
    if chave_encontrada:
        break

if chave_encontrada:
    col_ap, col_vg = chave_encontrada
    print(f"\n✅ Chave encontrada: aplicacoes['{col_ap}'] ↔ vagas['{col_vg}']")
    df_final = aplicacoes_df.merge(vagas_df, how='left', left_on=col_ap, right_on=col_vg, suffixes=('', '_vaga'))
    print(f"\n🟢 Merge realizado com sucesso. Shape final: {df_final.shape}")
else:
    print("\n❌ Nenhuma chave comum encontrada para merge. Verifique os dados manualmente.")


🔍 Possíveis chaves em 'aplicacoes': ['infos_basicas.codigo_profissional']
🔍 Possíveis chaves em 'vagas': ['id_vaga', 'informacoes_basicas.objetivo_vaga', 'informacoes_basicas.origem_vaga', 'informacoes_basicas.prioridade_vaga', 'informacoes_basicas.titulo_vaga', 'informacoes_basicas.vaga_sap', 'perfil_vaga.areas_atuacao', 'perfil_vaga.bairro', 'perfil_vaga.cidade', 'perfil_vaga.competencia_tecnicas_e_comportamentais', 'perfil_vaga.demais_observacoes', 'perfil_vaga.equipamentos_necessarios', 'perfil_vaga.estado', 'perfil_vaga.faixa_etaria', 'perfil_vaga.habilidades_comportamentais_necessarias', 'perfil_vaga.horario_trabalho', 'perfil_vaga.local_trabalho', 'perfil_vaga.nivel profissional', 'perfil_vaga.nivel_academico', 'perfil_vaga.nivel_espanhol', 'perfil_vaga.nivel_ingles', 'perfil_vaga.outro_idioma', 'perfil_vaga.pais', 'perfil_vaga.principais_atividades', 'perfil_vaga.regiao', 'perfil_vaga.vaga_especifica_para_pcd', 'perfil_vaga.viagens_requeridas']

✅ Chave encontrada: aplicacoes['

In [7]:
print(df_final.columns.tolist())

['id_aplicacao', 'cargo_atual.cargo_atual', 'cargo_atual.cliente', 'cargo_atual.data_admissao', 'cargo_atual.data_ultima_promocao', 'cargo_atual.email_corporativo', 'cargo_atual.email_superior_imediato', 'cargo_atual.id_ibrati', 'cargo_atual.nome_superior_imediato', 'cargo_atual.projeto_atual', 'cargo_atual.unidade', 'cv_en', 'cv_pt', 'formacao_e_idiomas.ano_conclusao', 'formacao_e_idiomas.cursos', 'formacao_e_idiomas.instituicao_ensino_superior', 'formacao_e_idiomas.nivel_academico', 'formacao_e_idiomas.nivel_espanhol', 'formacao_e_idiomas.nivel_ingles', 'formacao_e_idiomas.outro_curso', 'formacao_e_idiomas.outro_idioma', 'informacoes_pessoais.cpf', 'informacoes_pessoais.data_aceite', 'informacoes_pessoais.data_nascimento', 'informacoes_pessoais.download_cv', 'informacoes_pessoais.email', 'informacoes_pessoais.email_secundario', 'informacoes_pessoais.endereco', 'informacoes_pessoais.estado_civil', 'informacoes_pessoais.facebook', 'informacoes_pessoais.fonte_indicacao', 'informacoes_pe

In [8]:
[col for col in df_final.columns if 'status' in col.lower()]

[]

In [9]:
[col for col in df_final.columns if any(palavra in col.lower() for palavra in ['avaliacao', 'resultado', 'entrevista', 'motivo', 'decisao', 'andamento', 'final'])]

['informacoes_basicas.data_final']

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import joblib

colunas = [
    'formacao_e_idiomas.nivel_academico',
    'formacao_e_idiomas.nivel_ingles',
    'informacoes_profissionais.nivel_profissional',
    'informacoes_profissionais.remuneracao',
    'perfil_vaga.competencia_tecnicas_e_comportamentais',
    'informacoes_basicas.tipo_contratacao',
    'informacoes_basicas.cliente',
    'informacoes_basicas.data_final'
]

df_modelo = df_final[colunas].copy()
df_modelo['target'] = df_modelo['informacoes_basicas.data_final'].notna().astype(int)
df_modelo = df_modelo.drop(columns=['informacoes_basicas.data_final'])

X = df_modelo.drop('target', axis=1)
y = df_modelo['target']

le_dict = {}
for col in X.columns:
    if X[col].dtype == 'object':
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col].astype(str))
        le_dict[col] = le

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

print(classification_report(y_test, model.predict(X_test)))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      7582
           1       0.81      0.79      0.80       915

    accuracy                           0.96      8497
   macro avg       0.89      0.89      0.89      8497
weighted avg       0.96      0.96      0.96      8497



In [11]:
# Salvar modelo e recursos
joblib.dump(model, 'modelo_entrevista.pkl')
joblib.dump(le_dict, 'label_encoders.pkl')
joblib.dump(list(X.columns), 'features_utilizadas.pkl')


['features_utilizadas.pkl']

In [12]:
!pip install streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.3 MB/s eta 0:00:00


In [13]:
import joblib

model = joblib.load("modelo_entrevista.pkl")
le_dict = joblib.load("label_encoders.pkl")
features = joblib.load("features_utilizadas.pkl")

In [14]:
!pip install fastapi nest-asyncio pyngrok uvicorn

In [16]:
!ngrok config add-authtoken 2wpbIEbIm27HLtF6TFXUsCLDYrz_7BjGFnHfZ3Jr2iqaHsRpL

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [32]:
from fastapi import FastAPI, Request
from pydantic import BaseModel
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import logging

nest_asyncio.apply()

model = joblib.load("modelo_entrevista.pkl")
le_dict = joblib.load("label_encoders.pkl")
features = joblib.load("features_utilizadas.pkl")

logging.basicConfig(filename="logs_entrevistas.log", level=logging.INFO, format="%(asctime)s - %(message)s")

app = FastAPI(title="API Otimizacao de Entrevistas com IA")

@app.get("/")
def root():
    return {"mensagem": "API de Otimização de Entrevistas está no ar!"}

class Candidato(BaseModel):
    formacao_e_idiomas_nivel_academico: str
    formacao_e_idiomas_nivel_ingles: str
    informacoes_profissionais_nivel_profissional: str
    informacoes_profissionais_remuneracao: float
    perfil_vaga_competencia_tecnicas_e_comportamentais: str
    informacoes_basicas_tipo_contratacao: str
    informacoes_basicas_cliente: str

@app.post("/prever/")
def prever_conclusao(candidato: Candidato, request: Request):
    dados = pd.DataFrame([candidato.dict()])
    dados.columns = features

    for col in dados.columns:
        if col in le_dict:
            dados[col] = le_dict[col].transform(dados[col].astype(str))

    pred = model.predict(dados)[0]
    prob = model.predict_proba(dados)[0][pred]

    resultado = {
        "previsao": int(pred),
        "probabilidade": round(prob, 2),
        "mensagem": "Entrevista provável de ser concluída" if pred == 1 else "Entrevista não deve ser concluída"
    }

    logging.info(f"IP: {request.client.host} | Entrada: {candidato.dict()} | Resultado: {resultado}")
    return resultado

public_url = ngrok.connect(8000)
print("🔗 URL pública:", public_url)
print("📘 Documentação Swagger:", f"{public_url}/docs")

uvicorn.run(app, port=8000)

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-15' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    s

🔗 URL pública: NgrokTunnel: "https://fb8c-35-196-131-123.ngrok-free.app" -> "http://localhost:8000"
📘 Documentação Swagger: NgrokTunnel: "https://fb8c-35-196-131-123.ngrok-free.app" -> "http://localhost:8000"/docs


INFO:     Started server process [3303]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     177.27.247.165:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     177.27.247.165:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     177.27.247.165:0 - "POST /prever/ HTTP/1.1" 500 Internal Server Error


<ipython-input-32-b91688139547>:33: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  dados = pd.DataFrame([candidato.dict()])
ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/utils/_encode.py", line 235, in _encode
    return _map_to_integer(values, uniques)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/utils/_encode.py", line 174, in _map_to_integer
    return xp.asarray([table[v] for v in values], device=device(values))
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/utils/_encode.py", line 174, in <listcomp>
    return xp.asarray([table[v] for v in values], device=device(values))
                       ~~~~~^^^
  File "/usr/loc

INFO:     177.27.247.165:0 - "POST /prever/ HTTP/1.1" 500 Internal Server Error


<ipython-input-32-b91688139547>:33: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  dados = pd.DataFrame([candidato.dict()])
ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/utils/_encode.py", line 235, in _encode
    return _map_to_integer(values, uniques)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/utils/_encode.py", line 174, in _map_to_integer
    return xp.asarray([table[v] for v in values], device=device(values))
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/utils/_encode.py", line 174, in <listcomp>
    return xp.asarray([table[v] for v in values], device=device(values))
                       ~~~~~^^^
  File "/usr/loc

In [20]:
!pip install fastapi uvicorn nest_asyncio pyngrok

In [21]:
!pip install streamlit


In [76]:
import streamlit as st
import re
import json
import matplotlib.pyplot as plt

st.set_page_config(page_title="Painel IA de Entrevistas", layout="wide")
st.title("📊 Painel de Monitoramento - IA Otimização de Entrevistas")

LOG_FILE = "logs_entrevistas.log"

def parse_log(file_path):
    entradas = []
    if not os.path.exists(file_path):
        return pd.DataFrame()
    with open(file_path, "r") as f:
        for linha in f:
            try:
                match = re.search(r"IP: (.*?) \| Entrada: (.*?) \| Resultado: (.*)", linha)
                if match:
                    ip = match.group(1)
                    entrada = json.loads(match.group(2).replace("'", '"'))
                    resultado = json.loads(match.group(3).replace("'", '"'))
                    entrada.update(resultado)
                    entrada["Data"] = linha[:19]
                    entradas.append(entrada)
            except Exception as e:
                print(f"Erro ao processar linha: {e}")
    return pd.DataFrame(entradas)

df = parse_log(LOG_FILE)
if df.empty:
    st.warning("Nenhum log encontrado.")
else:
    col1, col2 = st.columns(2)
    with col1:
        filtro_pred = st.selectbox("🔍 Filtrar por previsão", options=["Todos", "Concluída", "Não concluída"])
        if filtro_pred != "Todos":
            df = df[df["previsao"] == (1 if filtro_pred == "Concluída" else 0)]
    with col2:
        qtd = st.slider("🔢 Quantidade de registros exibidos", min_value=5, max_value=100, value=20)

    st.markdown("### 📄 Registros Recentes")
    st.dataframe(df.sort_values("timestamp", ascending=False).head(qtd), use_container_width=True)

    st.markdown("---")
    st.markdown("### 📊 Distribuição das Previsões")
    contagem = df["previsao"].value_counts().sort_index()
    labels = ["Não Concluída", "Concluída"]
    valores = [contagem.get(0, 0), contagem.get(1, 0)]

    fig, ax = plt.subplots()
    ax.bar(labels, valores)
    ax.set_ylabel("Quantidade")
    ax.set_title("Previsões Registradas")
    st.pyplot(fig)


2025-05-21 19:07:19.695 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-21 19:07:19.697 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-21 19:07:19.698 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-21 19:07:19.701 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-21 19:07:19.702 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [78]:
log_linhas = [
    "2025-05-21 10:01:00 - IP: 127.0.0.1 | Entrada: {'formacao_e_idiomas_nivel_academico': 'Médio Completo', 'formacao_e_idiomas_nivel_ingles': 'Básico', 'informacoes_profissionais_nivel_profissional': 'Júnior', 'informacoes_profissionais_remuneracao': 2500.0, 'perfil_vaga_competencia_tecnicas_e_comportamentais': 'Atendimento, CRM', 'informacoes_basicas_tipo_contratacao': 'CLT', 'informacoes_basicas_cliente': 'Empresa A'} | Resultado: {'previsao': 0, 'probabilidade': 0.45, 'mensagem': 'Entrevista não deve ser concluída'}",
    "2025-05-21 10:15:23 - IP: 127.0.0.1 | Entrada: {'formacao_e_idiomas_nivel_academico': 'Superior Incompleto', 'formacao_e_idiomas_nivel_ingles': 'Intermediário', 'informacoes_profissionais_nivel_profissional': 'Pleno', 'informacoes_profissionais_remuneracao': 3200.0, 'perfil_vaga_competencia_tecnicas_e_comportamentais': 'Excel, Análise de Dados', 'informacoes_basicas_tipo_contratacao': 'PJ', 'informacoes_basicas_cliente': 'Empresa B'} | Resultado: {'previsao': 1, 'probabilidade': 0.76, 'mensagem': 'Entrevista provável de ser concluída'}",
    "2025-05-21 11:40:15 - IP: 127.0.0.1 | Entrada: {'formacao_e_idiomas_nivel_academico': 'Superior Completo', 'formacao_e_idiomas_nivel_ingles': 'Avançado', 'informacoes_profissionais_nivel_profissional': 'Sênior', 'informacoes_profissionais_remuneracao': 6000.0, 'perfil_vaga_competencia_tecnicas_e_comportamentais': 'Liderança, Python', 'informacoes_basicas_tipo_contratacao': 'CLT', 'informacoes_basicas_cliente': 'Empresa C'} | Resultado: {'previsao': 1, 'probabilidade': 0.91, 'mensagem': 'Entrevista provável de ser concluída'}",
    "2025-05-21 12:05:42 - IP: 127.0.0.1 | Entrada: {'formacao_e_idiomas_nivel_academico': 'Técnico', 'formacao_e_idiomas_nivel_ingles': 'Nenhum', 'informacoes_profissionais_nivel_profissional': 'Auxiliar', 'informacoes_profissionais_remuneracao': 1800.0, 'perfil_vaga_competencia_tecnicas_e_comportamentais': 'Manutenção', 'informacoes_basicas_tipo_contratacao': 'CLT', 'informacoes_basicas_cliente': 'Empresa D'} | Resultado: {'previsao': 0, 'probabilidade': 0.38, 'mensagem': 'Entrevista não deve ser concluída'}",
    "2025-05-21 13:20:00 - IP: 127.0.0.1 | Entrada: {'formacao_e_idiomas_nivel_academico': 'Pós-graduação', 'formacao_e_idiomas_nivel_ingles': 'Fluente', 'informacoes_profissionais_nivel_profissional': 'Especialista', 'informacoes_profissionais_remuneracao': 8500.0, 'perfil_vaga_competencia_tecnicas_e_comportamentais': 'Gestão, Estratégia', 'informacoes_basicas_tipo_contratacao': 'PJ', 'informacoes_basicas_cliente': 'Empresa E'} | Resultado: {'previsao': 1, 'probabilidade': 0.95, 'mensagem': 'Entrevista provável de ser concluída'}",
]
log_linhas = [
    "2025-05-21 10:01:00 - IP: 127.0.0.1 | Entrada: {'formacao_e_idiomas_nivel_academico': 'Médio Completo', 'formacao_e_idiomas_nivel_ingles': 'Básico', 'informacoes_profissionais_nivel_profissional': 'Júnior', 'informacoes_profissionais_remuneracao': 2500.0, 'perfil_vaga_competencia_tecnicas_e_comportamentais': 'Atendimento, CRM', 'informacoes_basicas_tipo_contratacao': 'CLT', 'informacoes_basicas_cliente': 'Empresa A'} | Resultado: {'previsao': 0, 'probabilidade': 0.45, 'mensagem': 'Entrevista não deve ser concluída'}",
    "2025-05-21 10:15:23 - IP: 127.0.0.1 | Entrada: {'formacao_e_idiomas_nivel_academico': 'Superior Incompleto', 'formacao_e_idiomas_nivel_ingles': 'Intermediário', 'informacoes_profissionais_nivel_profissional': 'Pleno', 'informacoes_profissionais_remuneracao': 3200.0, 'perfil_vaga_competencia_tecnicas_e_comportamentais': 'Excel, Análise de Dados', 'informacoes_basicas_tipo_contratacao': 'PJ', 'informacoes_basicas_cliente': 'Empresa B'} | Resultado: {'previsao': 1, 'probabilidade': 0.76, 'mensagem': 'Entrevista provável de ser concluída'}",
    "2025-05-21 11:40:15 - IP: 127.0.0.1 | Entrada: {'formacao_e_idiomas_nivel_academico': 'Superior Completo', 'formacao_e_idiomas_nivel_ingles': 'Avançado', 'informacoes_profissionais_nivel_profissional': 'Sênior', 'informacoes_profissionais_remuneracao': 6000.0, 'perfil_vaga_competencia_tecnicas_e_comportamentais': 'Liderança, Python', 'informacoes_basicas_tipo_contratacao': 'CLT', 'informacoes_basicas_cliente': 'Empresa C'} | Resultado: {'previsao': 1, 'probabilidade': 0.91, 'mensagem': 'Entrevista provável de ser concluída'}",
    "2025-05-21 12:05:42 - IP: 127.0.0.1 | Entrada: {'formacao_e_idiomas_nivel_academico': 'Técnico', 'formacao_e_idiomas_nivel_ingles': 'Nenhum', 'informacoes_profissionais_nivel_profissional': 'Auxiliar', 'informacoes_profissionais_remuneracao': 1800.0, 'perfil_vaga_competencia_tecnicas_e_comportamentais': 'Manutenção', 'informacoes_basicas_tipo_contratacao': 'CLT', 'informacoes_basicas_cliente': 'Empresa D'} | Resultado: {'previsao': 0, 'probabilidade': 0.38, 'mensagem': 'Entrevista não deve ser concluída'}",
    "2025-05-21 13:20:00 - IP: 127.0.0.1 | Entrada: {'formacao_e_idiomas_nivel_academico': 'Pós-graduação', 'formacao_e_idiomas_nivel_ingles': 'Fluente', 'informacoes_profissionais_nivel_profissional': 'Especialista', 'informacoes_profissionais_remuneracao': 8500.0, 'perfil_vaga_competencia_tecnicas_e_comportamentais': 'Gestão, Estratégia', 'informacoes_basicas_tipo_contratacao': 'PJ', 'informacoes_basicas_cliente': 'Empresa E'} | Resultado: {'previsao': 1, 'probabilidade': 0.95, 'mensagem': 'Entrevista provável de ser concluída'}",
]

with open("logs_entrevistas.log", "w", encoding="utf-8") as f:
    f.write("\n".join(log_linhas))

In [ ]:
!streamlit run painel.py &




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.196.131.123:8501



In [80]:
from pyngrok import ngrok

# Abre o túnel com protocolo explícito (novo formato exigido)
public_url = ngrok.connect("http://localhost:8501")
print("🔗 Painel disponível em:", public_url)


🔗 Painel disponível em: NgrokTunnel: "https://f0da-35-196-131-123.ngrok-free.app" -> "http://localhost:8501"


In [31]:
dockerfile = """
FROM python:3.9-slim

WORKDIR /app

COPY . .

RUN pip install --no-cache-dir -r requirements.txt

CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]
"""

with open("Dockerfile", "w") as f:
    f.write(dockerfile)

# Ver conteúdo
!cat Dockerfile



FROM python:3.9-slim

WORKDIR /app

COPY . .

RUN pip install --no-cache-dir -r requirements.txt

CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]


In [ ]:
!pip install pytest

In [ ]:
# Recriar o arquivo de testes unitários com tratamento robusto para valores não vistos

test_model_code = """\
import joblib
import pandas as pd

def test_model_carrega():
    model = joblib.load('modelo_entrevista.pkl')
    assert model is not None

def test_previsao_simples():
    model = joblib.load('modelo_entrevista.pkl')
    features = joblib.load('features_utilizadas.pkl')
    le_dict = joblib.load('label_encoders.pkl')

    dados = {
        'formacao_e_idiomas.nivel_academico': 'Ensino Médio',
        'formacao_e_idiomas.nivel_ingles': 'Intermediário',
        'informacoes_profissionais.nivel_profissional': 'Júnior',
        'informacoes_profissionais.remuneracao': 3500,
        'perfil_vaga.competencia_tecnicas_e_comportamentais': 'HTML, CSS',
        'informacoes_basicas.tipo_contratacao': 'CLT',
        'informacoes_basicas.cliente': 'Empresa XP'
    }

    df = pd.DataFrame([dados])
    for col in df.columns:
        if col in le_dict:
            encoder = le_dict[col]
            try:
                df[col] = encoder.transform([df[col][0]])
            except ValueError:
                df[col] = encoder.transform([encoder.classes_[0]])

    df = df[features]
    pred = model.predict(df)[0]
    assert pred in [0, 1]
"""

with open("test_model.py", "w") as f:
    f.write(test_model_code)

"/content/test_model.py criado com sucesso."

'/content/test_model.py criado com sucesso.'

In [ ]:
!pytest test_model.py

============================= test session starts ==============================
platform linux -- Python 3.11.12, pytest-8.3.5, pluggy-1.5.0
rootdir: /content
plugins: anyio-4.9.0, langsmith-0.3.39, typeguard-4.4.2
collected 2 items                                                              

test_model.py ..                                                         [100%]

============================== 2 passed in 1.52s ===============================


In [1]:
import logging
logging.basicConfig(level=logging.INFO)

In [ ]:
import sklearn
print(sklearn.__version__)

1.6.1


In [68]:
import os
print("Existe o log?", os.path.exists("logs_entrevistas.log"))

Existe o log? True
